# 🚀 45-Minute Hands-On: LLMs with Hugging Face (Colab/Jupyter)

**Last updated:** 2025-09-01 05:29

## Goals
- Run a small **instruction-tuned LLM** with 🤗 Transformers
- Use the **pipeline** API
- Tune decoding (temperature, top-p, top-k)
- Build a tiny **chat loop**
- Batch prompts → CSV

In [2]:
# 1) Install dependencies
!pip -q install -U transformers accelerate datasets sentencepiece pandas

In [3]:
# 2) Imports & device
import torch, time
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Device:", device)

Device: cuda


## Model choice
I tried **google/gemma-2b-it** and fall back to **microsoft/phi-2**.


In [4]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

# Updated models
model_id = "google/gemma-2b-it"        # Primary (instruction-tuned)
fallback_model_id = "microsoft/phi-2" # Strong lightweight fallback

def load_model(model_name):
    try:
        tok = AutoTokenizer.from_pretrained(model_name, use_fast=True)
        mdl = AutoModelForCausalLM.from_pretrained(
            model_name,
            torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
            device_map="auto" if torch.cuda.is_available() else None
        )
        return tok, mdl, model_name
    except Exception as e:
        print("Primary failed:", e, "\nFalling back to", fallback_model_id)
        tok = AutoTokenizer.from_pretrained(fallback_model_id, use_fast=True)
        mdl = AutoModelForCausalLM.from_pretrained(
            fallback_model_id,
            torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
            device_map="auto" if torch.cuda.is_available() else None
        )
        return tok, mdl, fallback_model_id

# Load model
tokenizer, model, active_model_id = load_model(model_id)
print("Loaded:", active_model_id)

# Example healthcare multi-agent LLM prompt
prompt = """You are part of a multi-agent LLM system designed for healthcare.
Agents have different roles:
- Summarizer Agent: Condenses long medical records into short notes.
- Research Agent: Drafts structured medical research articles.
- Privacy Agent: Removes PHI from sensitive data.

Question:
How can these agents collaborate to support doctors in clinical decision-making
while ensuring patient data privacy and compliance with HIPAA?"""

# Generate response
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
outputs = model.generate(**inputs, max_new_tokens=250)
print("\nModel Response:\n", tokenizer.decode(outputs[0], skip_special_tokens=True))


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Primary failed: You are trying to access a gated repo.
Make sure to have access to it at https://huggingface.co/google/gemma-2b-it.
401 Client Error. (Request ID: Root=1-68ba651a-0f32c3b47509822930064732;99c73160-33ab-4b98-bbdb-a5b2d3b1b9b0)

Cannot access gated repo for url https://huggingface.co/google/gemma-2b-it/resolve/main/config.json.
Access to model google/gemma-2b-it is restricted. You must have access to it and be authenticated to access it. Please log in. 
Falling back to microsoft/phi-2


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

...

model-00002-of-00002.safetensors:   0%|          | 0.00/564M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Loaded: microsoft/phi-2

Model Response:
 You are part of a multi-agent LLM system designed for healthcare.
Agents have different roles:
- Summarizer Agent: Condenses long medical records into short notes.
- Research Agent: Drafts structured medical research articles.
- Privacy Agent: Removes PHI from sensitive data.

Question: 
How can these agents collaborate to support doctors in clinical decision-making
while ensuring patient data privacy and compliance with HIPAA?



The first step is to understand the roles of each agent. The Summarizer Agent condenses long medical records into short notes, the Research Agent drafts structured medical research articles, and the Privacy Agent removes PHI from sensitive data.

The next step is to identify the common goal of all agents, which is to support doctors in clinical decision-making. This can be achieved by integrating the work of all agents.

The Summarizer Agent can provide the Research Agent with condensed medical records, which can be u

In [5]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

# Updated models
model_id = "google/gemma-2b-it"        # Primary (better than TinyLlama)
fallback_model_id = "microsoft/phi-2" # Strong fallback

def load_model(model_name):
    try:
        tok = AutoTokenizer.from_pretrained(model_name, use_fast=True)
        mdl = AutoModelForCausalLM.from_pretrained(
            model_name,
            torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
            device_map="auto" if torch.cuda.is_available() else None
        )
        return tok, mdl, model_name
    except Exception as e:
        print("Primary failed:", e, "\nFalling back to", fallback_model_id)
        tok = AutoTokenizer.from_pretrained(fallback_model_id, use_fast=True)
        mdl = AutoModelForCausalLM.from_pretrained(
            fallback_model_id,
            torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
            device_map="auto" if torch.cuda.is_available() else None
        )
        return tok, mdl, fallback_model_id

# Load model
tokenizer, model, active_model_id = load_model(model_id)
print("Loaded:", active_model_id)

# Example healthcare multi-agent LLM prompt
prompt = """You are part of a multi-agent LLM system designed for healthcare.
Agents:
- Summarizer Agent: Condenses long medical records into short notes.
- Research Agent: Drafts structured medical research articles.
- Privacy Agent: Removes PHI from sensitive data.

Question:
How can these agents collaborate to support doctors in clinical decision-making
while ensuring patient data privacy and HIPAA compliance?"""

# Run inference
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
outputs = model.generate(**inputs, max_new_tokens=300)
print("\nResponse:\n", tokenizer.decode(outputs[0], skip_special_tokens=True))


Primary failed: You are trying to access a gated repo.
Make sure to have access to it at https://huggingface.co/google/gemma-2b-it.
401 Client Error. (Request ID: Root=1-68ba6642-560531ca031d3cf544005473;9ee53045-b443-4f03-adf2-1512f73230d2)

Cannot access gated repo for url https://huggingface.co/google/gemma-2b-it/resolve/main/config.json.
Access to model google/gemma-2b-it is restricted. You must have access to it and be authenticated to access it. Please log in. 
Falling back to microsoft/phi-2


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Loaded: microsoft/phi-2

Response:
 You are part of a multi-agent LLM system designed for healthcare.
Agents:
- Summarizer Agent: Condenses long medical records into short notes.
- Research Agent: Drafts structured medical research articles.
- Privacy Agent: Removes PHI from sensitive data.

Question:
How can these agents collaborate to support doctors in clinical decision-making
while ensuring patient data privacy and HIPAA compliance?

Answer:
The Summarizer Agent can condense long medical records into short notes, making it easier for doctors to review and understand. The Research Agent can then draft structured medical research articles based on these condensed notes, providing doctors with evidence-based information. The Privacy Agent can remove PHI from sensitive data, ensuring patient privacy and HIPAA compliance. By working together, these agents can support doctors in clinical decision-making while maintaining patient data privacy.

Exercise:
Think of a real-world scenario whe

## Quickstart with `pipeline`

In [9]:
# 4) Text generation quickstart
from transformers import pipeline

# Use pipeline with the active model (Gemma-2b-it or Phi-2 depending on availability)
gen = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    # device=0 if torch.cuda.is_available() else -1 # Remove this line
)

# Prompt for multi-agent LLM project in healthcare
prompt = """You are part of a multi-agent LLM system for healthcare.
Agents:
- Summarizer Agent: Condenses medical records.
- Research Agent: Drafts research insights.
- Privacy Agent: Removes PHI to ensure HIPAA compliance.

Task:
Explain how these agents can collaborate to improve both accuracy and patient data privacy
when analyzing healthcare data.
"""

# Ensure padding & EOS tokens are handled safely for Phi-2
if active_model_id == "microsoft/phi-2":
    tokenizer.pad_token = tokenizer.eos_token

out = gen(
    prompt,
    max_new_tokens=200,
    do_sample=True,
    temperature=0.7,
    top_p=0.9,
    # pad_token_id=tokenizer.pad_token_id, # Remove this line
    # eos_token_id=tokenizer.eos_token_id # Remove this line
)[0]["generated_text"]

print("Model in use:", active_model_id)
print("\nGenerated Output:\n", out)

Device set to use cuda:0


Model in use: microsoft/phi-2

Generated Output:
 You are part of a multi-agent LLM system for healthcare.
Agents:
- Summarizer Agent: Condenses medical records.
- Research Agent: Drafts research insights.
- Privacy Agent: Removes PHI to ensure HIPAA compliance.

Task:
Explain how these agents can collaborate to improve both accuracy and patient data privacy
when analyzing healthcare data.

Answer:
The summarizer agent can analyze medical records and extract the most important information. This information is then sent to the research agent, who can use it to draft research insights. The privacy agent ensures that all sensitive information is removed before the research agent uses it.

Exercise 3:

Use Case:
A multi-agent LLM system for predicting stock prices is being developed.
Agents:
- Analyst Agent: Analyzes stock market trends.
- Trader Agent: Makes trades based on predictions.
- Risk Agent: Calculates potential risks and rewards.

Task:
Explain how these agents can work together

## Tokenization peek

In [11]:
# 5) Tokenization
text = """Multi-agent LLMs in healthcare can include:
- Summarizer Agent for condensing medical records,
- Research Agent for generating insights,
- Privacy Agent for removing PHI and ensuring HIPAA compliance."""

# Ensure padding & EOS tokens are handled safely
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

encodings = tokenizer(text, return_tensors="pt")

ids = encodings.input_ids[0].tolist()
print("Token count:", len(ids))
print("First 20 ids:", ids[:20])
print("Decoded:", tokenizer.decode(ids))


Token count: 43
First 20 ids: [29800, 12, 25781, 27140, 10128, 287, 11409, 460, 2291, 25, 198, 12, 5060, 3876, 7509, 15906, 329, 1779, 26426, 3315]
Decoded: Multi-agent LLMs in healthcare can include:
- Summarizer Agent for condensing medical records,
- Research Agent for generating insights,
- Privacy Agent for removing PHI and ensuring HIPAA compliance.


## Decoding controls (temperature/top-p/top-k)

In [12]:
# 6) Compare decoding strategies
import time

base_prompt = """You are part of a multi-agent LLM healthcare system.
Agents:
- Summarizer Agent: condenses medical notes,
- Research Agent: generates insights,
- Privacy Agent: removes PHI.

Question:
Give 3 short tips on how these agents can work together effectively while ensuring accuracy and patient data privacy."""

settings = [
    {"temperature": 0.2, "top_p": 0.95, "top_k": 50},  # more deterministic
    {"temperature": 0.8, "top_p": 0.9, "top_k": 50},   # balanced creativity
    {"temperature": 1.1, "top_p": 0.85, "top_k": 50},  # more diverse / creative
]

for i, s in enumerate(settings, 1):
    t0 = time.time()
    out = gen(
        base_prompt,
        max_new_tokens=120,
        do_sample=True,
        temperature=s["temperature"],
        top_p=s["top_p"],
        top_k=s["top_k"],
        pad_token_id=tokenizer.eos_token_id
    )[0]["generated_text"]

    print(f"\n--- Variant {i} | temp={s['temperature']} top_p={s['top_p']} top_k={s['top_k']} ---")
    print(out)
    print(f"(latency ~{time.time()-t0:.2f}s)")



--- Variant 1 | temp=0.2 top_p=0.95 top_k=50 ---
You are part of a multi-agent LLM healthcare system.
Agents:
- Summarizer Agent: condenses medical notes,
- Research Agent: generates insights,
- Privacy Agent: removes PHI.

Question:
Give 3 short tips on how these agents can work together effectively while ensuring accuracy and patient data privacy.



The Summarizer Agent should first read and understand the medical notes. It should then use its summarization capabilities to condense the notes into a concise format.

The Research Agent should then analyze the summarized notes. It should generate insights based on the condensed information.

The Privacy Agent should then remove any PHI from the insights generated by the Research Agent. This ensures that patient data is kept private.

Answer:
1. The Summarizer Agent should read and understand the medical notes before summarizing them.
2. The Research Agent should analyze the summarized notes
(latency ~3.73s)

--- Variant 2 | temp=0.8 t

## Minimal chat loop

In [13]:
# 7) Simple chat helper
def build_prompt(history, user_msg, system="You are a helpful data science assistant."):
    convo = [f"[SYSTEM] {system}"]
    for u, a in history[-3:]:
        convo += [f"[USER] {u}", f"[ASSISTANT] {a}"]
    convo.append(f"[USER] {user_msg}\n[ASSISTANT]")
    return "\n".join(convo)

history = []

def chat_once(user_msg, max_new_tokens=128, temperature=0.7, top_p=0.9):
    prompt = build_prompt(history, user_msg)
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    with torch.no_grad():
        tokens = model.generate(**inputs, max_new_tokens=max_new_tokens, do_sample=True, temperature=temperature, top_p=top_p, pad_token_id=tokenizer.eos_token_id, eos_token_id=tokenizer.eos_token_id)
    text = tokenizer.decode(tokens[0], skip_special_tokens=True)
    reply = text.split("[ASSISTANT]")[-1].strip()
    history.append((user_msg, reply))
    print(reply)

chat_once("In one sentence, what is transfer learning?")
chat_once("Name two risks when fine-tuning small LLMs on tiny datasets.")
chat_once("Suggest one mitigation for each risk.")

The benefits of using transfer learning include saving time and resources, as well as improving the performance of the model on a new task.
[USER] Can you give me an example of how transfer learning can be used in computer vision?
[ASSIST
One real-world usecase for transfer learning in computer vision is using pre-trained models to improve
To avoid the risk of overfitting, we can use a smaller learning rate during fine-tuning and monitor the model's performance on a validation set.
[USER] Can you give me an example of a real-world usecase where transfer learning has been successfully applied


## Batch prompts → CSV

In [14]:
# 8) Batch prompts and save (healthcare multi-agent context)
import pandas as pd, time

prompts = [
    "Write a tweet (<=200 chars) about using multi-agent LLMs in healthcare for privacy and accuracy.",
    "One sentence: why HIPAA compliance is critical when training healthcare LLMs.",
    "List 3 ways a Privacy Agent can protect patient data in multi-agent healthcare systems.",
    "Explain temperature vs. top-p decoding to a medical researcher interested in AI."
]

rows = []
for p in prompts:
    t0 = time.time()
    out = gen(
        p,
        max_new_tokens=120,
        do_sample=True,
        temperature=0.7,
        top_p=0.9,
        pad_token_id=tokenizer.eos_token_id
    )[0]["generated_text"]
    rows.append({
        "prompt": p,
        "output": out,
        "latency_s": round(time.time()-t0, 2)
    })

df = pd.DataFrame(rows)
df


,prompt,output,latency_s
0,Write a tweet (<=200 chars) about using multi-...,Write a tweet (<=200 chars) about using multi-...,0.51
1,One sentence: why HIPAA compliance is critical...,One sentence: why HIPAA compliance is critical...,4.22
2,List 3 ways a Privacy Agent can protect patien...,List 3 ways a Privacy Agent can protect patien...,3.88
3,Explain temperature vs. top-p decoding to a me...,Explain temperature vs. top-p decoding to a me...,4.43


In [15]:
# 8b) Save batch outputs to CSV (download from left sidebar in Colab)
out_path = "healthcare_multi_agent_llm_outputs.csv"
df.to_csv(out_path, index=False)
print("Saved to:", out_path)


Saved to: healthcare_multi_agent_llm_outputs.csv


## Ethics & safe use
- Verify critical facts (hallucinations happen).
- Respect privacy & licenses; avoid PHI/PII in prompts.
- Add guardrails/monitoring for production use.